<h4>Transfer Learning</h4>

<h5>Imports</h5>

In [12]:
from urllib.parse import urlparse, parse_qs, urlencode, urlunparse, unquote_plus
import tensorflow as tf
import numpy as np
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Embedding, GlobalAveragePooling1D, Input, Dropout
from tensorflow.keras.layers import TextVectorization
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import MultiHeadAttention, LayerNormalization, Layer
from tensorflow.keras.models import Model
from tensorflow.keras import layers, losses
import jsonlines
import os
import json
import pickle
from sklearn.metrics import f1_score, accuracy_score, precision_score
from typing import Callable
from sklearn import svm
from tensorflow.keras.optimizers import Adam
import tensorflow_probability as tfp
from tensorflow.python.profiler import profiler_v2 as profiler
import visualkeras
from PIL import ImageFont
import time
from sklearn.utils import shuffle
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, matthews_corrcoef, precision_recall_curve
import matplotlib.pyplot as plt
import math
import pickle

In [5]:
def parse_http_requests(file_path):
    requests = []
    current_request = {}
    with open(file_path, 'r') as file:
        lines = file.readlines()
        for line in lines:
            line = line.strip()
            if line == '':
                if current_request:
                    if (current_request['method'] == 'POST' or current_request['method'] == 'PUT') and 'body' not in current_request.keys():
                        continue    
                    requests.append(current_request)
                    current_request = {}
            elif line.startswith('GET') or line.startswith('POST') or line.startswith('PUT'):
                method, uri, _ = line.split(' ')
                current_request['method'] = method
                current_request['uri'] = uri
            else:
                if ': ' not in line:
                    current_request['body'] = line
                else:
                    header_name, header_value = line.split(': ', 1)
                    if 'headers' not in current_request:
                        current_request['headers'] = {}
                    current_request['headers'][header_name] = header_value
    if current_request:
        requests.append(current_request)
    return requests


file_paths = ['./dataset/cisc/normalTrafficTraining.txt', './dataset/cisc/anomalousTrafficTest.txt', './dataset/cisc/normalTrafficTest.txt']
cisc_train = parse_http_requests(file_paths[0])
cisc_test_malicious = parse_http_requests(file_paths[1])
cisc_test_normal = parse_http_requests(file_paths[2])

def parse_requests(requests):
    data=[]
    for request in requests:
        if request['method'] == 'POST':
            data.append('POST'+' '+request['uri']+' '+request['body']+' '+json.dumps(request['headers']))
        else:
            data.append('GET'+' '+request['uri']+' '+json.dumps(request['headers']))
    return data

train_examples = parse_requests(cisc_train)[:16000]
normal = parse_requests(cisc_test_normal)
malicious = parse_requests(cisc_test_malicious)
test_examples = normal+malicious
train_labels = [0] * len(train_examples)
test_labels = [0]* len(normal)
test_labels.extend([1] * len(malicious))

dataset_train = tf.data.Dataset.from_tensor_slices((list(map(lambda x: unquote_plus(x),train_examples)), train_labels))
dataset_test = tf.data.Dataset.from_tensor_slices((list(map(lambda x: unquote_plus(x),test_examples)), test_labels))

#shuflee datasets with given radnom seed
dataset_train = dataset_train.shuffle(400000, seed=42, reshuffle_each_iteration=False)
dataset_test = dataset_test.shuffle(400000, seed=42, reshuffle_each_iteration=False)

def preprocess_text(text, label):
    text = tf.strings.lower(text)
    text = tf.strings.regex_replace(text, "[a-zA-Z]+", "a")
    text = tf.strings.regex_replace(text, "[0-9]+", "n")
    text = tf.strings.regex_replace(text, "(a|n){2,}", "x")
    text = tf.strings.regex_replace(text, '[^\x00-\x7F]+', '')
    text = tf.strings.regex_replace(text, '(.)',  r'\1 ')
    return text, label

def preprocess_text_substitution(text, label):
    text = tf.strings.lower(text)
    text = tf.strings.regex_replace(text, "[a-zA-Z]+", "a")
    text = tf.strings.regex_replace(text, "[0-9]+", "n")
    text = tf.strings.regex_replace(text, "(a|n){2,}", "x")
    text = tf.strings.regex_replace(text, '[^\x00-\x7F]+', '')
    punctuation = "=?/(){}[]<>"
    for p in punctuation:
        text = tf.strings.regex_replace(text, "\\" + p, " "+p+" ")
    return text, label

 
# Map the preprocess function to the dataset
train = dataset_train.map(preprocess_text)
test = dataset_test.map(preprocess_text)

train_sub = dataset_train.map(preprocess_text_substitution)
test_sub = dataset_test.map(preprocess_text_substitution)

for text, label in train_sub.take(1):
    print(text.numpy(), label.numpy())

for text, label in train.take(1):
    print(text.numpy(), label.numpy())


b'a a: /  / a:n / x / a / a.a ? a = a a  { "a-a": "a / n.n  ( a; a / n.n; a )  a / n.n.n  ( a a ) ", "a": "a-a", "a-a": "a-a", "a": "a / a,a / a,a / a a,a / a;a = n.n,a / a;a = n.n,a / a,* / *;a = n.n", "a-a": "a-a, a-a, a, a", "a-a": "a-n, a-n;a = n.n, *;a = n.n", "a-a": "a", "a": "a:n", "a": "a = x", "a": "a" } ' 0
b'a   a : / / a : n / x / a / a . a ? a = a   a   { " a - a " :   " a / n . n   ( a ;   a / n . n ;   a )   a / n . n . n   ( a   a ) " ,   " a " :   " a - a " ,   " a - a " :   " a - a " ,   " a " :   " a / a , a / a , a / a   a , a / a ; a = n . n , a / a ; a = n . n , a / a , * / * ; a = n . n " ,   " a - a " :   " a - a ,   a - a ,   a ,   a " ,   " a - a " :   " a - n ,   a - n ; a = n . n ,   * ; a = n . n " ,   " a - a " :   " a " ,   " a " :   " a : n " ,   " a " :   " a = x " ,   " a " :   " a " } ' 0


2023-06-10 19:38:38.444740: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_1' with dtype int32 and shape [16000]
	 [[{{node Placeholder/_1}}]]
2023-06-10 19:38:38.477992: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_1' with dtype int32 and shape [16000]
	 [[{{node Placeholder/_1}}]]


In [6]:
len(train_examples)

16000

<h4>Save and load vectorization from disk</h4>

In [7]:

from_disk = pickle.load(open("vectorization.pkl", "rb"))
vectorization = TextVectorization.from_config(from_disk['config'])
# You have to call `adapt` with some dummy data (BUG in Keras)
vectorization.adapt(tf.data.Dataset.from_tensor_slices(["xyz"]))
vectorization.set_weights(from_disk['weights'])

from_disk_sub = pickle.load(open("vectorization_sub.pkl", "rb"))
vectorization_sub = TextVectorization.from_config(from_disk_sub['config'])
# You have to call `adapt` with some dummy data (BUG in Keras)
vectorization_sub.adapt(tf.data.Dataset.from_tensor_slices(["xyz"]))
vectorization_sub.set_weights(from_disk_sub['weights'])

2023-06-10 19:38:45.761960: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype string and shape [1]
	 [[{{node Placeholder/_0}}]]
2023-06-10 19:38:45.915681: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype string and shape [1]
	 [[{{node Placeholder/_0}}]]


<h4>Load and retrain all models</h4>

In [8]:
#load lstm model
lstm_model = tf.keras.models.load_model('lstm/my_model')
lstm_model_sub = tf.keras.models.load_model('lstm_sub/my_model')

#load transformer model
transformer_model = tf.keras.models.load_model('trans/my_model')
transformer_model_sub = tf.keras.models.load_model('trans_sub/my_model')

#load cnn model
cnn_model = tf.keras.models.load_model('cnn/my_model') 
cnn_model_sub = tf.keras.models.load_model('cnn_sub/my_model') 

#load autoencoder model
autoencoder_model = tf.keras.models.load_model('autoencoder/my_model')
autoencoder_model_sub = tf.keras.models.load_model('autoencoder_sub/my_model')

#create new svm model   
svm_model = svm.OneClassSVM(gamma='auto')
svm_model_sub = svm.OneClassSVM(gamma='auto')


2023-06-10 19:38:51.117526: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients_split_2_grad_concat_split_2_split_dim' with dtype int32
	 [[{{node gradients_split_2_grad_concat_split_2_split_dim}}]]
2023-06-10 19:38:51.117644: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients_split_grad_concat_split_split_dim' with dtype int32
	 [[{{node gradients_split_grad_concat_split_split_dim}}]]
2023-06-10 19:38:51.117697: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

2023-06-10 19:38:51.721538: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients_reversev2_grad_reversev2_reversev2_axis' with dtype int32 and shape [1]
	 [[{{node gradients_reversev2_grad_reversev2_reversev2_axis}}]]
2023-06-10 19:38:51.722752: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients_split_2_grad_concat_split_2_split_dim' with dtype int32
	 [[{{node gradients_split_2_grad_concat_split_2_split_dim}}]]
2023-06-10 19:38:51.722811: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message):

2023-06-10 19:38:52.202135: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients_split_2_grad_concat_split_2_split_dim' with dtype int32
	 [[{{node gradients_split_2_grad_concat_split_2_split_dim}}]]
2023-06-10 19:38:52.202246: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients_split_grad_concat_split_split_dim' with dtype int32
	 [[{{node gradients_split_grad_concat_split_split_dim}}]]
2023-06-10 19:38:52.202292: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

2023-06-10 19:38:52.410385: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients_reversev2_grad_reversev2_reversev2_axis' with dtype int32 and shape [1]
	 [[{{node gradients_reversev2_grad_reversev2_reversev2_axis}}]]
2023-06-10 19:38:52.411527: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients_split_2_grad_concat_split_2_split_dim' with dtype int32
	 [[{{node gradients_split_2_grad_concat_split_2_split_dim}}]]
2023-06-10 19:38:52.411587: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message):

2023-06-10 19:38:53.176684: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-06-10 19:38:53.177989: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-06-10 19:38:53.178880: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

2023-06-10 19:38:53.958029: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients_reversev2_grad_reversev2_reversev2_axis' with dtype int32 and shape [1]
	 [[{{node gradients_reversev2_grad_reversev2_reversev2_axis}}]]
2023-06-10 19:38:53.959171: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients_split_2_grad_concat_split_2_split_dim' with dtype int32
	 [[{{node gradients_split_2_grad_concat_split_2_split_dim}}]]
2023-06-10 19:38:53.959230: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message):

2023-06-10 19:38:54.330907: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients_reversev2_grad_reversev2_reversev2_axis' with dtype int32 and shape [1]
	 [[{{node gradients_reversev2_grad_reversev2_reversev2_axis}}]]
2023-06-10 19:38:54.332472: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients_split_2_grad_concat_split_2_split_dim' with dtype int32
	 [[{{node gradients_split_2_grad_concat_split_2_split_dim}}]]
2023-06-10 19:38:54.332559: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message):

2023-06-10 19:38:54.892300: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients_reversev2_grad_reversev2_reversev2_axis' with dtype int32 and shape [1]
	 [[{{node gradients_reversev2_grad_reversev2_reversev2_axis}}]]
2023-06-10 19:38:54.893367: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients_split_2_grad_concat_split_2_split_dim' with dtype int32
	 [[{{node gradients_split_2_grad_concat_split_2_split_dim}}]]
2023-06-10 19:38:54.893424: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message):

2023-06-10 19:38:55.159900: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients_reversev2_grad_reversev2_reversev2_axis' with dtype int32 and shape [1]
	 [[{{node gradients_reversev2_grad_reversev2_reversev2_axis}}]]
2023-06-10 19:38:55.161200: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients_split_2_grad_concat_split_2_split_dim' with dtype int32
	 [[{{node gradients_split_2_grad_concat_split_2_split_dim}}]]
2023-06-10 19:38:55.161261: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message):

In [11]:
#prepare train data
def sub_processing(text, label):
    return  vectorization_sub(text), label

def processing(text, label):
    return  vectorization(text), label

def autoencoder_processing(text, label):
    return  vectorization(text), vectorization(text)

def autoencoder_sub_processing(text, label):
    return  vectorization_sub(text), vectorization_sub(text)

auto_train_auto_sub = train_sub.map(autoencoder_sub_processing)
auto_train_auto = train.map(autoencoder_processing)

train_auto_sub = train_sub.map(sub_processing)
train_auto = train.map(processing)

# convert tf Dataset to numpy array
svm_train_auto = np.array(list(train_auto.as_numpy_iterator()))
svm_train_auto_sub = np.array(list(train_auto_sub.as_numpy_iterator()))

#extract the text and label from the numpy array
train_text = svm_train_auto[:,0]
train_label = svm_train_auto[:,1]
train_text_sub = svm_train_auto_sub[:,0]
train_label_sub = svm_train_auto_sub[:,1]

train_auto = train_auto.batch(32)
train_auto_sub = train_auto_sub.batch(32)

auto_train_auto_sub = auto_train_auto_sub.batch(32)
auto_train_auto = auto_train_auto.batch(32)

2023-06-10 19:45:52.645230: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_10' with dtype int64
	 [[{{node Placeholder/_10}}]]
/tmp/ipykernel_684/3845410109.py:21: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  svm_train_auto = np.array(list(train_auto.as_numpy_iterator()))
2023-06-10 19:45:55.763857: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_12' with dtype int64
	 [[{{

In [14]:
#retrain models and save
#cnn_model.fit(train_auto, epochs=10)
#cnn_model.save('cisc/cnn/my_model')
#cnn_model_sub.fit(train_auto_sub, epochs=10)
#cnn_model_sub.save('cisc/cnn_sub/my_model')

#transformer_model.fit(train_auto, epochs=10)
#transformer_model.save('cisc/transformer/my_model')
#transformer_model_sub.fit(train_auto_sub, epochs=10)
#transformer_model_sub.save('cisc/transformer_sub/my_model')

#lstm_model.fit(train_auto, epochs=10)
#lstm_model.save('cisc/lstm/my_model')
#lstm_model_sub.fit(train_auto_sub, epochs=10)
#lstm_model_sub.save('cisc/lstm_sub/my_model')

#autoencoder_model.fit(auto_train_auto, epochs=10)
#autoencoder_model.save('cisc/autoencoder/my_model')
#autoencoder_model_sub.fit(auto_train_auto_sub, epochs=10)
#autoencoder_model_sub.save('cisc/autoencoder_sub/my_model')

svm_model = svm_model.fit(list(train_text))
filename = './cisc/svm/svm_model.sav'
pickle.dump(svm_model, open(filename, 'wb'))
svm_model_sub = svm_model_sub.fit(list(train_text_sub))
filename = './cisc/svm_sub/svm_model.sav'
pickle.dump(svm_model_sub, open(filename, 'wb'))

In [ ]:
#load retrained models
cnn_model= tf.keras.models.load_model('cisc/cnn/my_model')
cnn_model_sub= tf.keras.models.load_model('cisc/cnn_sub/my_model')
transformer_model= tf.keras.models.load_model('cisc/transformer/my_model')
transformer_model_sub= tf.keras.models.load_model('cisc/transformer_sub/my_model')
lstm_model= tf.keras.models.load_model('cisc/lstm/my_model')
lstm_model_sub= tf.keras.models.load_model('cisc/lstm_sub/my_model')
autoencoder_model= tf.keras.models.load_model('cisc/autoencoder/my_model')
autoencoder_model_sub= tf.keras.models.load_model('cisc/autoencoder_sub/my_model')
filename = './cisc/svm/svm_model.sav'
svm_model=pickle.load( open(filename, 'rb'))
filename = './cisc/svm_sub/svm_model.sav'
svm_model_sub=pickle.load(open(filename, 'rb'))

In [15]:
#prepare test data
#testdataset for normal_models:
def sub_processing(text, label):
    return  vectorization_sub(text), label

def processing(text, label):
    return  vectorization(text), label
test_auto_sub = test_sub.map(sub_processing)

test_auto = test.map(processing)



#tesdataset for svm
test_auto_svm = np.array(list(test_auto.as_numpy_iterator()))
test_auto_sub_svm = np.array(list(test_auto_sub.as_numpy_iterator()))
test_text = test_auto_svm[:,0]
test_label = test_auto_svm[:,1]
test_text_sub = test_auto_sub_svm[:,0]
test_label_sub = test_auto_sub_svm[:,1]


test_auto_sub = test_auto_sub.batch(32)
test_auto = test_auto.batch(32)
def get_data(x, y):
  return x

# Use the map() function to extract the labels
dataset_tmp = test_auto.map(get_data)
dataset_tmp_sub = test_auto_sub.map(get_data)

2023-06-10 21:07:15.823871: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_1' with dtype int32 and shape [61065]
	 [[{{node Placeholder/_1}}]]
/tmp/ipykernel_684/1756571173.py:15: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  test_auto_svm = np.array(list(test_auto.as_numpy_iterator()))
2023-06-10 21:07:28.560520: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_1' with dtype

In [16]:
#run and time predictions
def predict_and_time(model,model_sub,name):
    print(f"Running predictions for {name}")
    reconstructions = model.predict(test_auto, verbose=0)
    reconstructions_sub = model_sub.predict(test_auto_sub, verbose=0)
    return reconstructions, reconstructions_sub
def predict_and_time_svm(model,model_sub):
    print(f"Running predictions for svm")
    reconstructions = model.predict(list(test_text))
    reconstructions_sub = model_sub.predict(list(test_text_sub)) 
    return reconstructions, reconstructions_sub    

reconstructions_cnn, reconstructions_sub_cnn = predict_and_time(cnn_model,cnn_model_sub,"cnn")
reconstructions_trans, reconstructions_sub_trans = predict_and_time(transformer_model,transformer_model_sub,"transformer")
reconstructions_lstm, reconstructions_sub_lstm = predict_and_time(lstm_model,lstm_model_sub,"lstm")
reconstructions_auto, reconstructions_sub_auto = predict_and_time(autoencoder_model,autoencoder_model_sub,"autoencoder")
reconstructions_svm, reconstructions_sub_svm = predict_and_time_svm(svm_model,svm_model_sub)


2023-06-10 21:07:40.512977: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_9' with dtype resource
	 [[{{node Placeholder/_9}}]]


Running predictions for cnn


2023-06-10 21:07:54.533961: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_1' with dtype int32 and shape [61065]
	 [[{{node Placeholder/_1}}]]


Running predictions for transformer
Running predictions for lstm


2023-06-10 21:08:37.286258: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-06-10 21:08:37.287636: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-06-10 21:08:37.288868: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

Running predictions for autoencoder
Running predictions for svm


In [17]:
def sigmoid(x):
  return 1 / (1 + math.exp(-x))

def calc_and_print_metrics(y_test,reconsturctions,name, mse_ratio=0.5,pred_treshold=0.5):
    
    if(name.find("auto") != -1):
        if(name.find("sub") != -1):
            tmp_sub = np.concatenate(list(dataset_tmp_sub.as_numpy_iterator()))
            mse_ = tf.keras.losses.mean_squared_error(tmp_sub, reconsturctions)
            mse = [0.01*x for x in mse_]
        else:  
            tmp = np.concatenate(list(dataset_tmp.as_numpy_iterator()))
            mse = tf.keras.losses.mean_squared_error(tmp, reconsturctions)
        avg =  sum(mse) / len(mse)
        print(avg)
        reconsturctions=list(map(lambda x: sigmoid(x.numpy()-mse_ratio*avg.numpy()), list(mse)))
        y_pred = list(map(lambda x: 0 if sigmoid(x.numpy()-mse_ratio*avg.numpy())<pred_treshold else 1, list(mse)))
    elif(name.find("svm") != -1):
        y_pred = list(map(lambda x: 1 if x==-1 else 0, list(reconsturctions)))
    else:
        y_pred = list(map(lambda x: 0 if x[0]<pred_treshold else 1, list(reconsturctions)))
    
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    mcc = matthews_corrcoef(y_test, y_pred)
    
    print(f"Metrics for model {name}")
    print("Accuracy:", accuracy)
    print("Precision:", precision)
    print("Recall:", recall)
    print("F1 Score:", f1)
    print("MCC:", mcc)

In [18]:
actual = test_auto.map(lambda x,y:y)
actual = np.concatenate(list(actual.as_numpy_iterator()))
calc_and_print_metrics(actual,reconstructions_cnn,"cnn",pred_treshold=0.1)
calc_and_print_metrics(actual,reconstructions_sub_cnn,"cnn_sub",pred_treshold=0.1)
calc_and_print_metrics(actual,reconstructions_trans,"trans",pred_treshold=0.1)
calc_and_print_metrics(actual,reconstructions_sub_trans,"trans_sub",pred_treshold=0.1)
calc_and_print_metrics(actual,reconstructions_lstm,"lstm",pred_treshold=0.1)
calc_and_print_metrics(actual,reconstructions_sub_lstm,"lstm_sub",pred_treshold=0.1)
calc_and_print_metrics(actual,reconstructions_auto,"auto",0.3)
calc_and_print_metrics(actual,reconstructions_sub_auto,"auto_sub", 0.3)
calc_and_print_metrics(actual,reconstructions_svm,"svm",pred_treshold=0.1)
calc_and_print_metrics(actual,reconstructions_sub_svm,"svm_sub",pred_treshold=0.1)

2023-06-10 21:11:55.357011: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype string and shape [61065]
	 [[{{node Placeholder/_0}}]]


KeyboardInterrupt: 